In [1]:
# Construction of dataset

import os, itertools, time, pickle, sys, glob, requests
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from nltk.corpus import wordnet
import tensorflow as tf
import tensorflow_hub as hub
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]

class Ontology():
    def __init__(self, ontology):
        self.ontology = ontology
        self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement
        self.construct_mapping_dict()
        
        self.subclasses = self.parse_subclasses()
        self.object_properties = self.parse_object_properties()
        self.data_properties = self.parse_data_properties()
        self.triples = self.parse_triples()
        self.classes = self.parse_classes()        
    
    def construct_mapping_dict(self):
        self.mapping_dict = {self.extract_ID(el, False): self.get_child_node(el, "rdfs:label")[0].firstChild.nodeValue for el in self.root.getElementsByTagName("owl:Class") if self.get_child_node(el, "rdfs:label")}
        self.mapping_dict_inv = {self.mapping_dict[key]: key for key in self.mapping_dict}
        return
        
    def get_child_node(self, element, tag):
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        return True if element.getAttribute(attribute).split("#")[-1] == value else False
    
    def get_subclass_triples(self):
        return [(b,a,c) for (a,b,c) in self.get_subclasses()]
    
    def parse_triples(self, union_flag=0, subclass_of=True):
        obj_props = self.object_properties
        data_props = self.data_properties
        props = obj_props + data_props
        all_triples = []
        for prop in props:
            domain_children = self.get_child_node(prop, "rdfs:domain")
            range_children = self.get_child_node(prop, "rdfs:range")
            domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children])
            range_prop = self.filter_null([self.extract_ID(el) for el in range_children])
            if not domain_children or not range_children:
                continue
#             if not domain_prop:
#                 domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children[0].getElementsByTagName("owl:Class")])
#             if not range_prop:
#                 range_prop = self.filter_null([self.extract_ID(el) for el in range_children[0].getElementsByTagName("owl:Class")])
            if domain_prop and range_prop:
                if union_flag == 0:
                    all_triples.extend([(el[0], el[1], self.extract_ID(prop)) for el in list(itertools.product(domain_prop, range_prop))])
                else:
                    all_triples.append(("###".join(domain_prop), "###".join(range_prop), self.extract_ID(prop)))
        if subclass_of:
            all_triples.extend(self.get_subclass_triples())
        return list(set(all_triples))
    
    def get_triples(self, union_flag=0, subclass_of=True):
        return self.parse_triples(union_flag, subclass_of)

    def parse_subclasses(self, union_flag=0):
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                subclass_pairs.append((el, el.parentNode, "subclass_of"))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if not level1_class:
                    continue
                else:
                    if self.extract_ID(level1_class[0]):
                        subclass_pairs.append((level1_class[0], el.parentNode, "subclass_of"))
                    else:
#                         level2classes = level1_class[0].getElementsByTagName("owl:Class")
#                         subclass_pairs.extend([(elem, el.parentNode, "subclass_of") for elem in level2classes if self.extract_ID(elem)])
                        continue
        return subclass_pairs
        
    def get_subclasses(self):
        subclasses = [(self.extract_ID(a), self.extract_ID(b), c) for (a,b,c) in self.subclasses]
        return [el for el in subclasses if el[0] and el[1] and el[2]]
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ID(self, element, check_coded = True):
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        element_id = element_id.split("#")[-1]
        if len(list(filter(str.isdigit, element_id))) >= 3 and "_" in element_id and check_coded:
            return self.mapping_dict[element_id]
        return element_id.replace("UNDEFINED_", "").replace("DO_", "")
    
    def parse_classes(self):
        class_elems = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        subclass_classes = list(set(flatten([el[:-1] for el in self.triples])))
        return list(set(self.filter_null(class_elems + subclass_classes)))
    
    def get_classes(self):
        return self.classes
    
    def get_entities(self):
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_data_properties(self):
        data_properties = [el for el in self.get_child_node(self.root, 'owl:DatatypeProperty')]
        fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_data_properties = [el for el in fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        inv_fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_data_properties = [el for el in inv_fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        return data_properties + fn_data_properties + inv_fn_data_properties
        
    def parse_object_properties(self):
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def get_object_properties(self):
        obj_props = [self.extract_ID(el) for el in self.object_properties]
        return list(set(self.filter_null(obj_props)))
    
    def get_data_properties(self):
        data_props = [self.extract_ID(el) for el in self.data_properties]
        return list(set(self.filter_null(data_props)))




In [3]:
USE_folder = "/home/vlead/USE"
alignment_folder = "reference-alignment/"

# Load reference alignments 
def load_alignments(folder):
    alignments = []
    for f in os.listdir(folder):
        doc = minidom.parse(folder + f)
        ls = list(zip(doc.getElementsByTagName('entity1'), doc.getElementsByTagName('entity2')))
        alignments.extend([(a.getAttribute('rdf:resource'), b.getAttribute('rdf:resource')) for (a,b) in ls])
    return alignments

# Extracting USE embeddings

def extractUSEEmbeddings(words):
    try:
        embed = hub.KerasLayer(USE_folder)
    except Exception as e:
        !mkdir $USE_folder
        !curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed" | tar -zxvC $USE_folder
        embed = hub.KerasLayer(USE_folder)
        pass
    word_embeddings = embed(words)
    return word_embeddings.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)


reference_alignments = load_alignments(alignment_folder)

# ra_anatomy_coded = load_alignments("../Anatomy/Alignments/")
# ra_anatomy = []
# ont1 = Ontology("../Anatomy/Ontologies/mouse.owl")
# ont2 = Ontology("../Anatomy/Ontologies/human.owl")
# for elem in ra_anatomy_coded:
#     pre1, pre2 = elem[0].split("#")[0].split(".")[0].split("/")[-1], elem[1].split("#")[0].split(".")[0].split("/")[-1]
#     elem1, elem2 = elem[0].split("#")[-1], elem[1].split("#")[-1]
#     ra_anatomy.append(( pre1 + "#" + ont1.mapping_dict[elem1], pre2 + "#" + ont2.mapping_dict[elem2]))

gt_mappings = [tuple([elem.split("/")[-1] for elem in el]) for el in reference_alignments]
# gt_mappings.extend(ra_anatomy)


ontologies_in_alignment = pickle.load(open("../data_generic.pkl", "rb"))[-1][:-1]
# ontologies_in_alignment += [["../Anatomy/Ontologies/human.owl", "../Anatomy/Ontologies/mouse.owl"]]


In [9]:
Ontology("conference_ontologies/iasted.owl").get_triples()

[('Activity', 'Item', 'is_connected_with'),
 ('Item', 'Person', 'is_sent_by'),
 ('Trip_day', 'Time', 'subclass_of'),
 ('Technic_activity', 'Conference_activity', 'subclass_of'),
 ('Review', 'Document', 'subclass_of'),
 ('Presentation', 'Conference_activity', 'subclass_of'),
 ('Conference_activity', 'Activity', 'subclass_of'),
 ('Introduction', 'Conference_activity', 'subclass_of'),
 ('Item', 'Activity', 'go_through'),
 ('One_day_presenter', 'Delegate', 'subclass_of'),
 ('Transport_vehicle', 'Item', 'subclass_of'),
 ('Credit_card', 'Card', 'subclass_of'),
 ('Person', 'Item', 'is_connected_with'),
 ('Person', 'State', 'occupy'),
 ('Video_cassette_player', 'Audiovisual_equipment', 'subclass_of'),
 ('Nonauthor_registration_fee', 'Registration_fee', 'subclass_of'),
 ('Social_program', 'Conference_activity', 'subclass_of'),
 ('Conference_hotel', 'Building', 'subclass_of'),
 ('Deadline', 'date', 'done_till'),
 ('Conference_airport', 'Building', 'subclass_of'),
 ('Submission', 'Document', 'sub

In [4]:
# Combinatorial mapping generation
entity_mappings, property_mappings = [], []

for l in ontologies_in_alignment:
    ont1 = Ontology("conference_ontologies/" + l[0] + ".owl")
    ont2 = Ontology("conference_ontologies/" + l[1] + ".owl")
    
    ent1 = ont1.get_entities()
    ent2 = ont2.get_entities()
    
    mappings = list(itertools.product(ent1, ent2))
    entity_mappings.extend([(l[0] + "#" + el[0], l[1] + "#" + el[1]) for el in mappings])
    
    obj1 = ont1.get_object_properties()
    obj2 = ont2.get_object_properties()
    
    data1 = ont1.get_data_properties()
    data2 = ont2.get_data_properties()
    
    mappings = list(itertools.product(obj1, obj2)) + list(itertools.product(data1, data2))
    property_mappings.extend([(l[0] + "#" + el[0], l[1] + "#" + el[1]) for el in mappings])
    
data_entity = {mapping: False for mapping in entity_mappings}
data_property = {mapping: False for mapping in property_mappings}

for mapping in set(gt_mappings):
    if mapping in data_entity:
        data_entity[mapping] = True
    elif mapping in data_property:
        data_property[mapping] = True
    else:
        print ("Error", mapping)


In [5]:
# Abbrevation resolution preprocessing

abbreviations_dict = {}
final_dict = {}

for mapping in entity_mappings + property_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_")])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_")])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}


left ('Chair_PC', 'Program_Chair') PC Program_Chair
left ('Chair_PC', 'Program_Committee_member') PCM Program_Committee
left ('Chair_PC', 'Program_Committee') PC Program_Committee
left ('Member_PC', 'Program_Chair') PC Program_Chair
left ('Member_PC', 'Program_Committee_member') PCM Program_Committee
left ('Member_PC', 'Program_Committee') PC Program_Committee
left ('Chair_PC', 'Presenter_city') PC Presenter_city
left ('Member_PC', 'Presenter_city') PC Presenter_city
left ('PC_Member', 'Program_Chair') PC Program_Chair
left ('PC_Member', 'Program_Committee_member') PCM Program_Committee
left ('PC_Member', 'Program_Committee') PC Program_Committee
left ('PC_Chair', 'Program_Chair') PC Program_Chair
left ('PC_Chair', 'Program_Committee_member') PCM Program_Committee
left ('PC_Chair', 'Program_Committee') PC Program_Committee
left ('OC_Member', 'Organizing_Committee') OC Organizing_Committee
left ('OC_Member', 'Organizing_Committee_member') OCM Organizing_Committee
left ('OC_Chair', 'Orga

In [7]:

def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

def parse(word):
    return flatten([el.split("_") for el in camel_case_split(word)])
    

extracted_elems = []

for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    entities = ont.get_entities()
    props = ont.get_object_properties() + ont.get_data_properties()
    triples = list(set(flatten(ont.get_triples())))
    extracted_elems.extend([ont_name + "#" + elem for elem in entities + props + triples])

extracted_elems = list(set(extracted_elems))
inp = [" ".join(parse(word.split("#")[1])) for word in extracted_elems]

# Lowering case except in abbreviations
inp_case_handled = []
for concept in inp:
    final_list = []
    for word in concept.split(" "):
        if not re.search("[A-Z][A-Z]+", concept):
            final_list.append(word.lower())
        else:
            final_list.append(word)
    case_resolved = " ".join(final_list)
    inp_case_handled.append(case_resolved)
inp = inp_case_handled

# Resolving abbreviations to full forms
inp_resolved = []
for concept in inp:
    for key in filtered_dict:
        concept = concept.replace(key, filtered_dict[key]).lower()
    inp_resolved.append(concept)

url = "https://montanaflynn-spellcheck.p.rapidapi.com/check/"

headers = {
    'x-rapidapi-host': "montanaflynn-spellcheck.p.rapidapi.com",
    'x-rapidapi-key': "9965b01207msh06291e57d6f2c55p1a6a16jsn0fb016da4a62"
    }

inp_spellchecked = []
for concept in inp_resolved:
    querystring = {"text": concept}
    response = requests.request("GET", url, headers=headers, params=querystring).json()
    if response["suggestion"] != concept:
        resolved = str(concept)
        final_list = []
        for word in concept.split(" "):
            if not re.search("[A-Z][A-Z]+", concept):
                final_list.append(word.lower())
            else:
                final_list.append(word)
        resolved = " ".join(final_list)
#         print (resolved, "suggestion", response)
        for word in response["corrections"]:
            if not re.search("[A-Z][A-Z]+", concept):
                resolved = resolved.replace(word.lower(), response["corrections"][word][0].lower())
                
        
        print (concept, resolved)
        inp_spellchecked.append(resolved)
    else:
        inp_spellchecked.append(concept)

print ("Total number of extracted unique classes and properties from entire RA set: ", len(extracted_elems))

extracted_elems = ["<UNK>"] + extracted_elems

# stopwords = ["has", "is", "a", "an", "the"]
stopwords = ["has"]
inp_stemmed = []
for elem in inp_spellchecked:
    words = " ".join([word for word in elem.split() if word not in stopwords])
    words = words.replace("-", " ")
    inp_stemmed.append(words)


embeds = np.array([np.zeros(512,)] + list(extractUSEEmbeddings(inp_stemmed)))
# embeds = np.array([np.zeros(512,)] + list(extractUSEEmbeddings(inp_spellchecked)))
embeddings = dict(zip(extracted_elems, embeds))


emb_vals = list(embeddings.values())
emb_indexer = {key: i for i, key in enumerate(list(embeddings.keys()))}
emb_indexer_inv = {i: key for i, key in enumerate(list(embeddings.keys()))}



modelling modeling
iasted non member tasted non member
organises organists
acm sigkdd acm sigrid
tprogram committee member program committee member
cheque exchequer
has programme has programmer
hardcopy mailing manifests printed by hard copy mailing manifests printed by
oc member co member
scientifically organises scientifically organists
memeber registration fee member registration fee
has a url has a url
sc member sc member
scientifically organised by scientifically organized by
holded by holed by
conference www conference www
contribution 1th-author contribution th-author
author attendee cd registration fee author attendee cd registration fee
programme programmer
flyer flayer
registration sigkdd member registration sigrid member
reviewes reviewed
has a commtitee has a committee
initial manuscipt initial manuscript
computer networks aapplications topic computer networks applications topic
programme brochure programmer brochure
oc chair co chair
technically organises technically organ

In [85]:
def path_to_root(elem, ont_mappings):
    if elem not in ont_mappings or not ont_mappings[elem]:
        return []
    output = ont_mappings[elem] + path_to_root(ont_mappings[elem][0], ont_mappings)
    return output

def find_path_to_root(ont):
    ont_obj = Ontology("conference_ontologies/" + ont + ".owl")
    triples = ont_obj.get_triples()
    entities = [(a,b) for (a,b,c) in triples]
    properties = [c for (a,b,c) in triples]
    
    entities_neighbours_dict = {elem: [] for elem in list(set(flatten(entities)))}
#     entities_neighbours_dict["Thing"] = []
    properties_neighbours_dict = {elem: [] for elem in list(set(properties))}
    
    for e1, e2, p in triples:
        if p=="subclass_of" and e2!="Thing":
            entities_neighbours_dict[e1].append(e2)
        else:
            properties_neighbours_dict[p].append((e1, e2))
    
#     for elem in entities_neighbours_dict:
#         if not entities_neighbours_dict[elem] and elem!="Thing":
#             entities_neighbours_dict[elem] = ["Thing"]
            
    entities_neighbours_dict = {elem: [elem] + path_to_root(elem, entities_neighbours_dict) for elem in entities_neighbours_dict}

#     if ont == "edas":
#         print (entities_neighbours_dict)
    
#     prop_triples = ont_obj.get_triples(subclass_of=False)
#     neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
#     for e1, e2, p in prop_triples:
#         neighbours_dict_props[p].extend([e1, e2])

    #neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    # for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
    #     if elem not in neighbours_dict:
    #         neighbours_dict[elem] = [elem]

#     entities_neighbours_dict = {el: entities_neighbours_dict[el][:1] + sorted(list(set(entities_neighbours_dict[el][1:])))
#                        for el in entities_neighbours_dict}
#     neighbours_dict = {el: neighbours_dict[el][:23] for el in neighbours_dict if len( neighbours_dict[el]) > 2}
#     entities_neighbours_dict = {ont + "#" + el: [ont + "#" + e for e in entities_neighbours_dict[el]] for el in entities_neighbours_dict if len(entities_neighbours_dict[el]) == 1}
    entities_neighbours_dict = {ont + "#" + el: [ont + "#" + e for e in entities_neighbours_dict[el]] for el in entities_neighbours_dict if len(entities_neighbours_dict[el]) > 1}
    
    properties_neighbours_dict = {ont + "#" + el: [tuple([ont + "#" + e for e in tup]) for tup in properties_neighbours_dict[el]] for el in properties_neighbours_dict if len(properties_neighbours_dict[el]) == 1}

    return entities_neighbours_dict

neighbours_dicts = {ont: find_path_to_root(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
max_neighbours = np.max(flatten([[len(el[e]) for e in el] for el in neighbours_dicts.values()]))
neighbours_lens = {ont: {key: len(neighbours_dicts[ont][key]) for key in neighbours_dicts[ont]}
                   for ont in neighbours_dicts}
# neighbours_dicts = {ont: {key: neighbours_dicts[ont][key] + ["<UNK>" for i in range(max_neighbours -len(neighbours_dicts[ont][key]))]
#               for key in neighbours_dicts[ont]} for ont in neighbours_dicts}

# # ontologies_in_alignment = [[el.split("/")[1].split(".")[0] for el in ont] for ont in ontologies_in_alignment]
f = open("data_path.pkl", "wb")
pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts, ontologies_in_alignment], f)


In [67]:
data = {**data_entity, **data_property}

In [73]:
neighbours_dicts["confOf"]

{'confOf#Administrative_event': ['confOf#Administrative_event',
  'confOf#Event',
  '<UNK>',
  '<UNK>',
  '<UNK>',
  '<UNK>',
  '<UNK>'],
 'confOf#Registration_of_participants_event': ['confOf#Registration_of_participants_event',
  'confOf#Administrative_event',
  'confOf#Event',
  '<UNK>',
  '<UNK>',
  '<UNK>',
  '<UNK>'],
 'confOf#Workshop': ['confOf#Workshop',
  'confOf#Working_event',
  'confOf#Event',
  '<UNK>',
  '<UNK>',
  '<UNK>',
  '<UNK>'],
 'confOf#Submission_event': ['confOf#Submission_event',
  'confOf#Administrative_event',
  'confOf#Event',
  '<UNK>',
  '<UNK>',
  '<UNK>',
  '<UNK>'],
 'confOf#Participant': ['confOf#Participant',
  'confOf#Person',
  '<UNK>',
  '<UNK>',
  '<UNK>',
  '<UNK>',
  '<UNK>'],
 'confOf#Social_event': ['confOf#Social_event',
  'confOf#Event',
  '<UNK>',
  '<UNK>',
  '<UNK>',
  '<UNK>',
  '<UNK>'],
 'confOf#Assistant': ['confOf#Assistant',
  'confOf#Person',
  '<UNK>',
  '<UNK>',
  '<UNK>',
  '<UNK>',
  '<UNK>'],
 'confOf#Chair_PC': ['confOf#Chai

In [28]:
cos_sim(*extractUSEEmbeddings(['late paid applicant', 'late registered participant']))

0.6148052215576172

In [9]:
# AML test
def is_test(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) in test_onto

results = []
all_ont_pairs = list(set([tuple([el.split("#")[0] for el in l]) for l in data.keys()]))
for i in list(range(0, len(all_ont_pairs), 3)):
    test_onto = all_ont_pairs[i:i+3]
    for ont_pair in test_onto:
        a, b, c = ont_pair[0], ont_pair[1], ont_pair[0] + "-" + ont_pair[1]
        java_command = "java -jar AML_v3.1/AgreementMakerLight.jar -s conference_ontologies/" + a + ".owl" + \
                            " -t conference_ontologies/" + b + ".owl -o AML-test-results/" + c + ".rdf -a"
        process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    print (os.listdir("AML-test-results/"))
    pred_aml = load_alignments("AML-test-results/")
    pred_aml = [tuple([el.split("/")[-1] for el in key]) for key in pred_aml]
    tp = len([elem for elem in pred_aml if data[elem]])
    fn = len([key for key in gt_mappings if key not in set(pred_aml) and is_test(test_onto, key)])
    fp = len([elem for elem in pred_aml if not data[elem]])

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1score = 2 * precision * recall / (precision + recall)
    f2score = 5 * precision * recall / (4 * precision + recall)
    f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    print (precision, recall, f1score, f2score, f0_5score)
    
    metrics = [precision, recall, f1score, f2score, f0_5score]
    results.append(metrics)
    
    _ = [os.remove(f) for f in glob.glob('AML-test-results/*')]
    
print ("Final Results:", np.mean(results, axis=0))

['confOf-sigkdd.rdf', 'iasted-sigkdd.rdf', 'cmt-ekaw.rdf']
0.8275862068965517 0.7272727272727273 0.7741935483870968 0.7453416149068324 0.8053691275167786
['confOf-iasted.rdf', 'conference-edas.rdf', 'cmt-sigkdd.rdf']
0.8148148148148148 0.5789473684210527 0.6769230769230768 0.6145251396648045 0.7534246575342465
['ekaw-sigkdd.rdf', 'conference-sigkdd.rdf', 'conference-confOf.rdf']
0.78125 0.6097560975609756 0.684931506849315 0.6377551020408163 0.7396449704142012
['confOf-edas.rdf', 'edas-iasted.rdf', 'cmt-conference.rdf']
0.7941176470588235 0.5094339622641509 0.6206896551724137 0.548780487804878 0.7142857142857143
['edas-sigkdd.rdf', 'conference-iasted.rdf', 'ekaw-iasted.rdf']
0.7916666666666666 0.48717948717948717 0.6031746031746031 0.5277777777777778 0.7037037037037036
['cmt-confOf.rdf', 'cmt-edas.rdf', 'edas-ekaw.rdf']
0.8181818181818182 0.5192307692307693 0.6352941176470589 0.5601659751037344 0.733695652173913
['conference-ekaw.rdf', 'confOf-ekaw.rdf', 'cmt-iasted.rdf']
0.79545454545

In [8]:
for i in list(range(0, len(ontologies_in_alignment)-1, 3)):
    
    test_onto = ontologies_in_alignment[i:i+3]
    print (test_onto)

[['conference', 'ekaw'], ['confOf', 'ekaw'], ['ekaw', 'sigkdd']]
[['edas', 'sigkdd'], ['confOf', 'sigkdd'], ['iasted', 'sigkdd']]
[['confOf', 'iasted'], ['conference', 'iasted'], ['confOf', 'edas']]
[['edas', 'iasted'], ['conference', 'edas'], ['cmt', 'ekaw']]
[['cmt', 'confOf'], ['cmt', 'edas'], ['conference', 'sigkdd']]
[['cmt', 'sigkdd'], ['conference', 'confOf'], ['edas', 'ekaw']]
[['cmt', 'conference'], ['cmt', 'iasted'], ['ekaw', 'iasted']]


In [34]:
Ontology("conference_ontologies/cmt.owl").triples

[('Paper', 'Document', 'subclass_of'),
 ('Reviewer', 'ConferenceMember', 'subclass_of'),
 ('Conference', 'Administrator', 'virtualMeetingEnabledBy'),
 ('Administrator', 'Conference', 'startReviewerBidding'),
 ('Reviewer', 'User', 'subclass_of'),
 ('ConferenceChair', 'ConferenceMember', 'subclass_of'),
 ('Paper', 'Bid', 'hasBid'),
 ('Conference', 'Administrator', 'detailsEnteredBy'),
 ('Rejection', 'Decision', 'subclass_of'),
 ('Meta-Review', 'Review', 'subclass_of'),
 ('ExternalReviewer', 'Reviewer', 'assignedByReviewer'),
 ('Administrator', 'Conference', 'enableVirtualMeeting'),
 ('Administrator', 'ProgramCommitteeMember', 'setMaxPapers'),
 ('Co-author', 'Paper', 'co-writePaper'),
 ('Conference', 'boolean', 'acceptsHardcopySubmissions'),
 ('ProgramCommittee', 'ProgramCommitteeMember', 'hasProgramCommitteeMember'),
 ('Paper', 'unsignedLong', 'paperID'),
 ('Author', 'Paper', 'writePaper'),
 ('Paper', 'Co-author', 'hasCo-author'),
 ('Paper', 'string', 'title'),
 ('Author', 'Paper', 'subm

In [20]:
def count_non_unk(elem):
    return len([l for l in elem if l!="<UNK>"])
neighbours_dicts = {ont: {el: neighbours_dicts[ont][el][:int(sys.argv[1])] for el in neighbours_dicts[ont]
       if count_non_unk(neighbours_dicts[ont][el]) > int(sys.argv[2])} for ont in neighbours_dicts}

(167, 1240)

In [18]:
from collections import Counter
c = Counter(dict(sorted(Counter(flatten([l.split() for l in inp])).items(), key=lambda x:x[1], reverse=True)))
c

Counter({'has': 95,
         'paper': 61,
         'conference': 58,
         'of': 54,
         'is': 49,
         'by': 46,
         'topic': 39,
         'a': 35,
         'review': 31,
         'event': 27,
         'committee': 27,
         'member': 24,
         'chair': 22,
         'registration': 18,
         'date': 18,
         'author': 18,
         'reviewer': 15,
         'for': 14,
         'session': 14,
         'fee': 13,
         'contribution': 13,
         'program': 13,
         'speaker': 12,
         'abstract': 11,
         'name': 11,
         'computer': 11,
         'deadline': 11,
         'submission': 10,
         'city': 9,
         'hotel': 9,
         'in': 9,
         'to': 9,
         'networks': 9,
         'student': 9,
         'was': 8,
         'document': 8,
         'presenter': 8,
         'an': 8,
         'time': 8,
         'presentation': 8,
         'on': 8,
         'call': 7,
         'subclass': 7,
         'invited': 7,
         'tal

In [65]:
fn, fp = pickle.load(open("test_v2.pkl", "rb"))

In [83]:
import requests

url = "https://montanaflynn-spellcheck.p.rapidapi.com/check/"

headers = {
    'x-rapidapi-host': "montanaflynn-spellcheck.p.rapidapi.com",
    'x-rapidapi-key': "9965b01207msh06291e57d6f2c55p1a6a16jsn0fb016da4a62"
    }

# inp_spellchecked = []
for concept in inp[731:]:
    querystring = {"text": concept}
    response = requests.request("GET", url, headers=headers, params=querystring).json()
    if response["suggestion"] != concept:
        resolved = str(concept)
        for word in response["corrections"]:
            if not re.search("[A-Z][A-Z]+", concept):
                resolved = resolved.replace(word, response["corrections"][word][0])
        
        inp_spellchecked.append(resolved)
        print (concept, resolved)
    else:
        inp_spellchecked.append(concept)




registeered applicant registered applicant
technically organised by technically organized by
ngo no
sponzorship sponsorship


In [77]:
querystring = {"text": "technically Organised By"}
response = requests.request("GET", url, headers=headers, params=querystring)
response.json()

{'original': 'technically Organised By',
 'suggestion': 'technically Organized By',
 'corrections': {'Organised': ['Organized',
   'Organist',
   'Organism',
   'Organizes',
   'Disorganize',
   'Organize',
   'Agonized']}}

In [27]:
gt_mappings

[('conference#Information_for_participants', 'ekaw#Programme_Brochure'),
 ('conference#Person', 'ekaw#Person'),
 ('conference#Tutorial', 'ekaw#Tutorial'),
 ('conference#Review', 'ekaw#Review'),
 ('conference#has_a_review', 'ekaw#hasReview'),
 ('conference#Workshop', 'ekaw#Workshop'),
 ('conference#Late_paid_applicant', 'ekaw#Late-Registered_Participant'),
 ('conference#Early_paid_applicant', 'ekaw#Early-Registered_Participant'),
 ('conference#Organization', 'ekaw#Organisation'),
 ('conference#Track-workshop_chair', 'ekaw#Workshop_Chair'),
 ('conference#Abstract', 'ekaw#Abstract'),
 ('conference#Conference_proceedings', 'ekaw#Conference_Proceedings'),
 ('conference#Conference_volume', 'ekaw#Conference'),
 ('conference#Rejected_contribution', 'ekaw#Rejected_Paper'),
 ('conference#Poster', 'ekaw#Poster_Paper'),
 ('conference#Track', 'ekaw#Track'),
 ('conference#Topic', 'ekaw#Research_Topic'),
 ('conference#Conference_www', 'ekaw#Web_Site'),
 ('conference#Invited_speaker', 'ekaw#Invited_Sp

In [78]:
fn_spellchecked, fp_spellchecked = [dict(el) for el in pickle.load(open("test_v2.pkl", "rb"))]
fn_baseline, fp_baseline = [dict(el) for el in pickle.load(open("test_best.pkl", "rb"))]
fn_unhas, fp_unhas = [dict(el) for el in pickle.load(open("test_unhas.pkl", "rb"))]
fn_resolved, fp_resolved = [dict(el) for el in pickle.load(open("test_resolved.pkl", "rb"))]

fn_dict, fp_dict = {}, {}
def create_comparison_file(file, idx):
    fn, fp = [dict(el) for el in pickle.load(open(file, "rb"))]
    
    for key in fn:
        if key in fn_dict:
            fn_dict[key][idx] = fn[key]
        else:
            fn_dict[key] = ["N/A" for i in range(4)]
            fn_dict[key][idx] = fn[key]
    
    for key in fp:
        if key in fp_dict:
            fp_dict[key][idx] = fp[key]
        else:
            fp_dict[key] = ["N/A" for i in range(4)]
            fp_dict[key][idx] = fp[key]
    

create_comparison_file("test_best.pkl", 0)
create_comparison_file("test_unhas.pkl", 1)
create_comparison_file("test_v2.pkl", 2)
create_comparison_file("test_resolved.pkl", 3)

open("fn - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fn_dict.items()]))
open("fp - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fp_dict.items()]))

7796

[(('confOf#Member_PC', 'sigkdd#Program_Committee_member'), 0.460159499824062),
 (('confOf#Chair_PC', 'sigkdd#Program_Chair'), 0.710800728376894),
 (('iasted#Student_registration_fee', 'sigkdd#Registration_Student'),
  0.8369251770941893),
 (('iasted#Deadline_for_notification_of_acceptance',
   'sigkdd#Deadline_Author_notification'),
  0.6263288918181976),
 (('iasted#Nonmember_registration_fee', 'sigkdd#Registration_Non-Member'),
  0.8778317801581551),
 (('cmt#ConferenceMember', 'ekaw#Conference_Participant'), 0.5669564770296416),
 (('cmt#Author', 'ekaw#Paper_Author'), 0.8141944477873271),
 (('cmt#writtenBy', 'ekaw#reviewWrittenBy'), 0.7592434303979944),
 (('cmt#hasBeenAssigned', 'ekaw#reviewerOfPaper'), 0.2502871547766108),
 (('cmt#assignedTo', 'ekaw#hasReviewer'), 0.3499445900379554),
 (('cmt#PaperFullVersion', 'ekaw#Regular_Paper'), 0.5529287816442782),
 (('confOf#Administrative_event', 'iasted#Activity_before_conference'),
  0.3867995449037805),
 (('confOf#Reception', 'iasted#Coctai

In [72]:
d = {('confOf#Organization', 'sigkdd#Organizator'): (1,2,3,4),
 ('iasted#Document', 'sigkdd#Document'): (5,6,78,8)}
[[str(el) for el in flatten(el)] for el in d.items()]

[['confOf#Organization', 'sigkdd#Organizator', '1', '2', '3', '4'],
 ['iasted#Document', 'sigkdd#Document', '5', '6', '78', '8']]

In [34]:
abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_")])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_")])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}
inp_resolved = []
for concept in inp:
    for key in filtered_dict:
        concept = concept.replace(key, filtered_dict[key])
    inp_resolved.append(concept)
inp_resolved

left ('Chair_PC', 'Program_Chair') PC Program_Chair
left ('Chair_PC', 'Program_Committee') PC Program_Committee
left ('Chair_PC', 'Program_Committee_member') PCM Program_Committee
left ('Member_PC', 'Program_Chair') PC Program_Chair
left ('Member_PC', 'Program_Committee') PC Program_Committee
left ('Member_PC', 'Program_Committee_member') PCM Program_Committee
left ('Chair_PC', 'Presenter_city') PC Presenter_city
left ('Member_PC', 'Presenter_city') PC Presenter_city
left ('OC_Member', 'Organizing_Committee_member') OCM Organizing_Committee
left ('OC_Member', 'Organizing_Committee') OC Organizing_Committee
left ('PC_Member', 'Program_Chair') PC Program_Chair
left ('PC_Member', 'Program_Committee') PC Program_Committee
left ('PC_Member', 'Program_Committee_member') PCM Program_Committee
left ('OC_Chair', 'Organizing_Committee_member') OCM Organizing_Committee
left ('OC_Chair', 'Organizing_Committee') OC Organizing_Committee
left ('PC_Chair', 'Program_Chair') PC Program_Chair
left ('PC_C

In [48]:
keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))


In [25]:
cos_sim(*extractUSEEmbeddings(["Deadline for notification of acceptance", "Deadline Author notification"]))

0.6263289451599121

In [74]:
fn, fp = pickle.load(open("test_path5_1.pkl", "rb"))

In [76]:
ontologies_in_alignment

[['confOf', 'sigkdd'],
 ['iasted', 'sigkdd'],
 ['cmt', 'ekaw'],
 ['confOf', 'iasted'],
 ['conference', 'edas'],
 ['cmt', 'sigkdd'],
 ['ekaw', 'sigkdd'],
 ['conference', 'confOf'],
 ['conference', 'sigkdd'],
 ['confOf', 'edas'],
 ['cmt', 'conference'],
 ['edas', 'iasted'],
 ['conference', 'iasted'],
 ['edas', 'sigkdd'],
 ['ekaw', 'iasted'],
 ['cmt', 'edas'],
 ['edas', 'ekaw'],
 ['cmt', 'confOf'],
 ['confOf', 'ekaw'],
 ['conference', 'ekaw'],
 ['cmt', 'iasted']]

In [79]:
fp

[(('cmt#ProgramCommitteeChair', 'ekaw#PC_Chair'), 0.9999934369224981),
 (('cmt#ProgramCommitteeMember', 'ekaw#PC_Member'), 0.9999839542611472),
 (('iasted#Hotel_room', 'sigkdd#Hotel'), 0.9455056507783086),
 (('confOf#Organization', 'sigkdd#Organizator'), 1.0),
 (('cmt#writtenBy', 'ekaw#writtenBy'), 1.0),
 (('iasted#pay', 'sigkdd#pay'), 1.0),
 (('iasted#Document', 'sigkdd#Document'), 1.0),
 (('iasted#obtain', 'sigkdd#obtain'), 1.0),
 (('cmt#Chairman', 'sigkdd#General_Chair'), 0.9182169664306405),
 (('confOf#Social_event', 'iasted#Social_program'), 0.9243420630169328),
 (('conference#Passive_conference_participant', 'edas#Attendee'),
  0.9045829570298654),
 (('cmt#PaperAbstract', 'sigkdd#Abstract'), 0.9398814618172477),
 (('conference#Presentation', 'edas#PaperPresentation'), 0.9227664359315235),
 (('cmt#Author', 'sigkdd#Author_of_paper'), 0.895869406686101),
 (('conference#Conference', 'edas#Conference'), 1.0),
 (('cmt#date', 'sigkdd#Date'), 1.0),
 (('cmt#name', 'sigkdd#Name'), 1.0),
 (

In [23]:
Ontology("conference_ontologies/conference.owl").triples

[('Conference_proceedings', 'string', 'has_a_name'),
 ('Committee_member', 'Committee', 'was_a_member_of'),
 ('Review', 'Conference_document', 'subclass_of'),
 ('Review_preference', 'Reviewer', 'belongs_to_reviewers'),
 ('Information_for_participants', 'Conference_document', 'subclass_of'),
 ('Regular_author', 'Conference_contributor', 'subclass_of'),
 ('Late_paid_applicant', 'Paid_applicant', 'subclass_of'),
 ('Active_conference_participant', 'Presentation', 'gives_presentations'),
 ('Invited_speaker', 'Conference_contributor', 'subclass_of'),
 ('Program_committee', 'Conference_volume', 'was_a_program_committee_of'),
 ('Topic', 'Review_preference', 'has_been_assigned_a_review_reference'),
 ('Conference_volume', 'Important_dates', 'has_important_dates'),
 ('Rejected_contribution', 'Reviewed_contribution', 'subclass_of'),
 ('Contribution_co-author', 'Regular_author', 'subclass_of'),
 ('Active_conference_participant', 'Conference_contributor', 'subclass_of'),
 ('Presentation', 'Abstract'

In [81]:
fn, fp = pickle.load(open("test_lstm4_2.pkl", "rb"))

In [83]:
fn

[(('confOf#Member_PC', 'sigkdd#Program_Committee_member'), 0.9821488151234276),
 (('confOf#hasEmail', 'sigkdd#E-mail'), 0.9161555776735063),
 (('confOf#Chair_PC', 'sigkdd#Program_Chair'), 0.7888411974945077),
 (('iasted#Student_registration_fee', 'sigkdd#Registration_Student'),
  0.4687132189112725),
 (('iasted#Deadline_for_notification_of_acceptance',
   'sigkdd#Deadline_Author_notification'),
  0.6263288918181976),
 (('iasted#Nonmember_registration_fee', 'sigkdd#Registration_Non-Member'),
  0.6004082945181627),
 (('iasted#Author', 'sigkdd#Author'), 0.3705828543501226),
 (('cmt#ConferenceMember', 'ekaw#Conference_Participant'), 0.834017866174724),
 (('cmt#Author', 'ekaw#Paper_Author'), 0.8141944477873271),
 (('cmt#writtenBy', 'ekaw#reviewWrittenBy'), 0.7592434303979944),
 (('cmt#hasBeenAssigned', 'ekaw#reviewerOfPaper'), 0.2502871547766108),
 (('cmt#assignedTo', 'ekaw#hasReviewer'), 0.3499445900379554),
 (('cmt#PaperFullVersion', 'ekaw#Regular_Paper'), 0.10331508386734323),
 (('confOf

In [84]:
fp

[(('iasted#obtain', 'sigkdd#obtain'), 1.0),
 (('iasted#pay', 'sigkdd#pay'), 1.0),
 (('iasted#Document', 'sigkdd#Document'), 1.0),
 (('confOf#Organization', 'sigkdd#Organizator'), 1.0),
 (('cmt#writtenBy', 'ekaw#writtenBy'), 1.0),
 (('confOf#Camera_Ready_event', 'iasted#Receiving_manuscript'),
  0.9066686556244711),
 (('cmt#Administrator', 'sigkdd#Program_Chair'), 0.9217834141598852),
 (('conference#Chair', 'edas#Reception'), 0.981828811095803),
 (('conference#Registeered_applicant', 'edas#ComputerNetworksOpticalTopic'),
  0.996343065443642),
 (('confOf#Registration_of_participants_event', 'iasted#Lecture'),
  0.9902831965079355),
 (('conference#Passive_conference_participant', 'edas#FreeTimeBreak'),
  0.9415719517377226),
 (('confOf#Banquet', 'iasted#Author'), 0.9932710473256426),
 (('conference#Conference', 'edas#Conference'), 1.0),
 (('cmt#name', 'sigkdd#Name'), 1.0),
 (('cmt#date', 'sigkdd#Date'), 1.0),
 (('conference#Conference', 'confOf#Conference'), 1.0),
 (('ekaw#Organisation', 